In [ ]:
appId = '' 
appSecret = '' 
tenantId = ''

# Playing with "Windows Defender ATP"
- Related to: Microsoft Defender for Endpoint (MDE) (previously called Microsoft Advanced Threat Protection- MDATP) 
- Documentation: https://docs.microsoft.com/en-us/microsoft-365/security/defender-endpoint/apis-intro
- Resource URI: https://api.securitycenter.windows.com
- API required: Windows Defender ATP
- API permissions required:
  - AdvancedQuery.Read.All
  - Alert.Read.All
  - File.Read.All
  - Domains.Read.All
  - IP.Read.All
  - Machine.Read.All
  - Indicator.Read.All
  - User.Read.All
  - Score.Read.All
  - Software.Read.All
  - Vunerability.Read.All
  - Recommendation.Read.All 
- How to request access to admin: $https://login.microsoftonline.com/common/oauth2/authorize?prompt=consent&client_id=<HERE-THE-CLIENT-ID>&response_type=code&sso_reload=true$
- List of exposed APIs: https://docs.microsoft.com/en-us/microsoft-365/security/defender-endpoint/exposed-apis-list

# Required libraries

In [ ]:
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_rows", None)

import json
import urllib.parse

<h1 align='center'>========================================================<br>=======================================================</h1>

# Function to get the token to access via the app

In [ ]:
def mde_getaadtoken(tenantId, appId, appSecret):
    url = "https://login.windows.net/%s/oauth2/token" % (tenantId)
    resourceAppIdUri = 'https://api.securitycenter.windows.com'
    body = {
        'resource' : resourceAppIdUri,
        'client_id' : appId,
        'client_secret' : appSecret,
        'grant_type' : 'client_credentials'
    }
    data = urllib.parse.urlencode(body).encode("utf-8")
    req = urllib.request.Request(url, data)
    response = urllib.request.urlopen(req)
    jsonResponse = json.loads(response.read())
    aadToken = jsonResponse["access_token"]
    return aadToken

# Instantiating previous function

In [ ]:
access_token = mde_getaadtoken(tenantId, appId, appSecret)

<h1 align='center'>========================================================<br>=======================================================<br> ADVANCED HUNTING API</h1>

Note: this API is the same as the of from the M365D. Examples of queries [here!](https://github.com/microsoft/Microsoft-365-Defender-Hunting-Queries)

# Function Run (KQL)
https://docs.microsoft.com/en-us/windows/security/threat-protection/microsoft-defender-atp/run-advanced-query-api

In [ ]:
def mde_advancedquery_run(access_token,kql_query):
    url = "https://api.securitycenter.microsoft.com/api/advancedqueries/run"
    req = urllib.request.Request(url, 
                                 headers={'Authorization' : "Bearer " + access_token, 'Content-Type':'application/json'},
                                 data=bytes(json.dumps({"Query": kql_query}), encoding="utf-8"))
    response = urllib.request.urlopen(req)
    jsonResponse = json.loads(response.read())
    return pd.DataFrame.from_records(jsonResponse['Results'])

# Example of instantiating this function

In [ ]:
kql_query = """
DeviceProcessEvents
|limit 10
"""

In [ ]:
df_mde_advancedquery_run = mde_advancedquery_run(access_token,kql_query)
df_mde_advancedquery_run.head()

<h1 align='center'>========================================================<br>=======================================================<br>ALERT API</h1>

List of methods in the Alert API (https://docs.microsoft.com/en-us/windows/security/threat-protection/microsoft-defender-atp/alerts)

|Method	| Return Type |Description  |
|:- |:-|:-|
|Get alert|	Alert|	Get a single alert object.|
|List alerts|	Alert| collection	List alert collection.|
|Update alert|	Alert|	Update specific alert.|
|Create alert|	Alert|	Create an alert based on event data obtained from Advanced Hunting.|
|List related domains|	Domain collection|	List URLs associated with the alert.|
|List related files|	File collection|	List the file entities that are associated with the alert.|
|List related IPs|	IP collection|	List IPs that are associated with the alert.|
|Get related machines|	Machine|	The machine that is associated with the alert.|
|Get related users|	User|	The user that is associated with the alert.|

<h1 align='center'>...........................................................</h1>

# FUNCTION List alerts

In [ ]:
def mde_alerts_listalerts(access_token):
    url = "https://api.securitycenter.windows.com/api/alerts"
    req = urllib.request.Request(url, headers={'Authorization' : "Bearer " + access_token})
    response = urllib.request.urlopen(req)
    jsonResponse = json.loads(response.read())
    return pd.DataFrame.from_records(jsonResponse['value'])

# Instantiating the above function

In [ ]:
df_alerts = mde_alerts_listalerts(access_token)

# How the list of alerts looks like?

In [ ]:
df_alerts.head()

<h1 align='center'>...........................................................</h1>

# FUNCTION Get alert

In [ ]:
def mde_alerts_getalert(access_token,alert_id):
    url = "https://api.securitycenter.windows.com/api/alerts/"+str(alert_id)
    req = urllib.request.Request(url, headers={'Authorization' : "Bearer " + access_token})
    response = urllib.request.urlopen(req)
    jsonResponse = json.loads(response.read())
    return pd.DataFrame.from_dict(jsonResponse,orient='index').transpose()

# Instantiating the previous function

In [ ]:
mde_alerts_getalert(access_token,df_alerts['id'][0])

<h1 align='center'>...........................................................</h1>

# FUNCTION List related domains

In [ ]:
def mde_alerts_listrelateddomains(access_token,alarm_id):
    url = "https://api.securitycenter.windows.com/api/alerts/"+str(alarm_id)+"/domains"
    req = urllib.request.Request(url, headers={'Authorization' : "Bearer " + access_token})
    response = urllib.request.urlopen(req)
    jsonResponse = json.loads(response.read())
    return pd.DataFrame.from_records(jsonResponse['value'])

# Instantiating the previous function

In [ ]:
mde_alerts_listrelateddomains(access_token,df_alerts['id'][0])

<h1 align='center'>...........................................................</h1>


# FUNCTION List related files

In [ ]:
def mde_alerts_listrelatedfiles(access_token,alert_id):
    url = "https://api.securitycenter.windows.com/api/alerts/"+str(alert_id)+"/files"
    req = urllib.request.Request(url, headers={'Authorization' : "Bearer " + access_token})
    response = urllib.request.urlopen(req)
    jsonResponse = json.loads(response.read())
    return pd.DataFrame.from_records(jsonResponse['value'])

# Instantiating the previous function

In [ ]:
mde_alerts_listrelatedfiles(access_token,df_alerts['id'][0])

<h1 align='center'>...........................................................</h1>

# FUNCTION List related IPs

In [ ]:
def mde_alerts_listrelatedips(access_token,alarm_id):
    url = "https://api.securitycenter.windows.com/api/alerts/"+str(alarm_id)+"/ips"
    req = urllib.request.Request(url, headers={'Authorization' : "Bearer " + access_token})
    response = urllib.request.urlopen(req)
    jsonResponse = json.loads(response.read())
    return pd.DataFrame.from_records(jsonResponse['value'])

# Instantiating the previous function

In [ ]:
mde_alerts_listrelatedips(access_token,df_alerts['id'][0])

<h1 align='center'>...........................................................</h1>

# FUNCTION Get related machines

In [ ]:
def mde_alerts_getrelatedmachines(access_token,alarm_id):
    url = "https://api.securitycenter.windows.com/api/alerts/"+str(alarm_id)+"/machine"
    req = urllib.request.Request(url, headers={'Authorization' : "Bearer " + access_token})
    response = urllib.request.urlopen(req)
    jsonResponse = json.loads(response.read())
    return pd.DataFrame.from_dict(jsonResponse,orient='index').transpose()

# Instantiating the previous function

In [ ]:
mde_alerts_getrelatedmachines(access_token,df_alerts['id'][0])

<h1 align='center'>========================================================<br>=======================================================</h1>

# List of methods in the Machine API
https://docs.microsoft.com/en-us/windows/security/threat-protection/microsoft-defender-atp/machine

|Method|	Return Type|	Description|
|:- |:-|:-|
|List machines|	machine collection|	List set of machine entities in the org.|
|Get machine|	machine	|Get a machine by its identity.|
|Get logged on users|	user collection|	Get the set of User that logged on to the machine.|
|Get related alerts	|alert collection|	Get the set of alert entities that were raised on the machine.|
|Get installed software|	software collection|	Retrieves a collection of installed software related to a given machine ID.|
|Get discovered vulnerabilities|	vulnerability collection|	Retrieves a collection of discovered vulnerabilities related to a given machine ID.|
|Get security recommendations|	recommendation collection|	Retrieves a collection of security recommendations related to a given machine ID.|
|Add or Remove machine tags|	machine|	Add or Remove tag to a specific machine.|
|Find machines by IP|	machine collection|	Find machines seen with IP.|
|Find machines by tag|	machine collection|	Find machines by Tag.|
|Get missing KBs|	KB collection|	Get a list of missing KBs associated with the machine ID.|
|Set device value|	machine collection|	Set the value of a device.|

<h1 align='center'>...........................................................</h1>

# FUNCTION List machines

In [ ]:
def mde_machines_listmachines(access_token):
    url = "https://api.securitycenter.microsoft.com/api/machines"
    req = urllib.request.Request(url, headers={'Authorization' : "Bearer " + access_token})
    response = urllib.request.urlopen(req)
    jsonResponse = json.loads(response.read())
    return pd.DataFrame.from_records(jsonResponse['value'])

# Instantiating the previous function

In [ ]:
df_machines = mde_machines_listmachines(access_token)
df_machines

<h1 align='center'>...........................................................</h1>

# FUNCTION Get machines

In [ ]:
def mde_machines_getmachine(access_token,machine_id):
    url = "https://api.securitycenter.windows.com/api/machines/"+str(machine_id)
    req = urllib.request.Request(url, headers={'Authorization' : "Bearer " + access_token})
    response = urllib.request.urlopen(req)
    jsonResponse = json.loads(response.read())
    return pd.DataFrame.from_dict(jsonResponse,orient='index').transpose()

# Instantiating the previous function

In [ ]:
mde_machines_getmachine(access_token,df_machines['id'][0])